In [60]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width' , 1000)

In [61]:
# Variables
season = "2020-21"

In [62]:
all_players = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_players.csv', parse_dates= ["news_added"])
all_teams = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv' , parse_dates = ['kickoff_time'])
player_future_fixture = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])




In [63]:
all_teams.head()

,team_code,draw,form,team_id,loss,team_name,played,points,position,team_short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,NaN,1,0,Arsenal,0,0,0,ARS,4,NaN,False,0,1190,1270,1190,1200,1160,1270,1
1,7,0,NaN,2,0,Aston Villa,0,0,0,AVL,2,NaN,False,0,1050,1080,1010,1080,1000,1110,2
2,36,0,NaN,3,0,Brighton,0,0,0,BHA,2,NaN,False,0,1070,1070,1100,1130,1060,1060,131
3,90,0,NaN,4,0,Burnley,0,0,0,BUR,3,NaN,False,0,1140,1150,1130,1240,1050,1170,43
4,8,0,NaN,5,0,Chelsea,0,0,0,CHE,4,NaN,False,0,1200,1260,1140,1270,1240,1250,4


In [64]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,ep_this,event_points,form,player_id,in_dreamteam,news,news_added,now_cost,photo,PPG,selected_by_percent,status,team_id,team_code,total_points,...,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,PP90,VAPM,PPMM
0,NaN,NaN,37605,0,0,0,0,0,3,3.4,NaN,0,0.0,1,False,NaN,NaT,70,37605.jpg,2.9,2.1,a,1,3,53,...,5,20,0,0,0,1,0,0,1,256,223.6,582.9,190.0,99.8,253,109,35,29,174,95,135,72,3.314802,0.128571,0.414286
1,NaN,NaN,39476,0,0,0,0,0,2,3.1,NaN,0,0.0,2,False,NaN,NaT,50,39476.jpg,3.0,0.6,a,1,3,57,...,4,25,0,0,0,6,0,0,5,305,436.2,36.8,110.0,58.5,144,53,293,105,222,64,233,74,3.024764,0.200000,0.600000
2,NaN,NaN,41270,0,0,0,0,0,2,3.5,NaN,0,0.0,3,False,NaN,NaT,55,41270.jpg,2.8,1.7,a,1,3,94,...,8,42,0,0,0,5,2,0,10,494,701.6,106.7,211.0,102.1,50,18,221,61,160,32,130,35,3.011748,0.145455,0.509091
3,NaN,NaN,54694,0,0,0,0,0,3,5.0,NaN,0,0.0,4,False,NaN,NaT,120,54694.jpg,5.7,32.9,a,1,3,205,...,10,44,0,0,0,3,1,0,37,807,1006.0,479.6,1369.0,285.2,8,4,54,42,9,3,11,7,5.883291,0.308333,0.475000
4,NaN,NaN,58822,0,0,0,0,0,2,3.1,NaN,0,0.0,5,False,NaN,NaT,50,58822.jpg,2.9,0.4,a,1,3,61,...,4,20,0,0,0,1,0,0,3,286,349.0,218.9,118.0,68.7,182,68,158,34,221,63,201,59,3.535093,0.180000,0.580000


In [65]:
df_merge = all_teams[["team_code" , "strength" , "team_short_name" , "strength_overall_home" , "strength_overall_away"]]
all_players = all_players.merge(df_merge, on = 'team_code')

df_merge = player_types[['player_type_id' , 'position_name_short']]
all_players = all_players.merge(df_merge , on = "player_type_id")


#merged_data.drop(["element"], axis = 1, inplace = True)
#merged_data.rename(columns = {"fixture" : "fixture_id" , "opponent_team" : "opponent_team_id" , "round" : "gameweek_id" , "value" : "player_price", "short_name" : "opponent_team_name" , "strength" : "opponent_strength"}, inplace = True)

#merged_data.sort_values("gameweek_id" , ascending = True , inplace = True)



In [66]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'player_code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'player_type_id', 'ep_next', 'ep_this', 'event_points', 'form', 'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'PPG', 'selected_by_percent', 'status', 'team_id', 'team_code', 'total_points', 'transfers_in_event', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'threat_rank', 'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type', 'PP90', 'VAPM', 'PPMM', 'strength', 'team_short_name', 'strength_overall_home', 'strength_overall_away',
     

In [67]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,ep_this,event_points,form,player_id,in_dreamteam,news,news_added,now_cost,photo,PPG,selected_by_percent,status,team_id,team_code,total_points,...,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,PP90,VAPM,PPMM,strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,NaN,NaN,37605,0,0,0,0,0,3,3.4,NaN,0,0.0,1,False,NaN,NaT,70,37605.jpg,2.9,2.1,a,1,3,53,...,1,0,0,1,256,223.6,582.9,190.0,99.8,253,109,35,29,174,95,135,72,3.314802,0.128571,0.414286,4,ARS,1190,1270,MID
1,NaN,NaN,54694,0,0,0,0,0,3,5.0,NaN,0,0.0,4,False,NaN,NaT,120,54694.jpg,5.7,32.9,a,1,3,205,...,3,1,0,37,807,1006.0,479.6,1369.0,285.2,8,4,54,42,9,3,11,7,5.883291,0.308333,0.475000,4,ARS,1190,1270,MID
2,NaN,NaN,84450,0,0,0,0,0,3,2.8,NaN,0,0.0,9,False,NaN,NaT,55,84450.jpg,2.4,0.4,a,1,3,73,...,10,0,0,2,408,486.2,298.5,110.0,89.9,125,46,119,77,223,114,153,81,2.540603,0.072727,0.436364,4,ARS,1190,1270,MID
3,NaN,NaN,195735,0,0,0,0,0,3,3.7,NaN,0,0.0,18,False,NaN,NaT,80,195735.jpg,3.4,2.4,a,1,3,106,...,4,0,0,9,353,500.2,639.4,624.0,176.5,120,42,27,21,49,26,45,25,4.760479,0.175000,0.425000,4,ARS,1190,1270,MID
4,NaN,NaN,198849,0,0,0,0,0,3,2.4,NaN,0,0.0,19,False,NaN,NaT,50,198849.jpg,1.6,0.2,a,1,3,46,...,7,0,0,0,247,281.8,207.1,207.0,69.7,215,90,164,107,165,89,198,99,2.758161,-0.080000,0.320000,4,ARS,1190,1270,MID


## Save to file

In [68]:
all_players.to_csv (r'./data/' + season + '/Merged_data/all_players_merged.csv', index = False, header=True)
